In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests as re
import xmltodict as xml2dict
from collections import OrderedDict

%matplotlib inline

In [34]:
url = 'http://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ListarProposicoes'
params = {
    'sigla':'',
    'numero':'',
    'ano':'',
    'datApresentacaoIni':'',
    'datApresentacaoFim':'',
    'parteNomeAutor':'',
    'idTipoAutor':'',
    'siglaPartidoAutor':'',
    'siglaUFAutor':'',
    'generoAutor':'',
    'codEstado':'',
    'codOrgaoEstado':'',
    'emTramitacao':''
}

In [35]:
params['sigla'] = 'PL'
params['ano'] = '2017'
response = re.get(url, params)
print('URL: %s\nStatus: %s' % (response.url, response.status_code))

URL: http://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ListarProposicoes?generoAutor=&siglaUFAutor=&ano=2017&numero=&codOrgaoEstado=&emTramitacao=&datApresentacaoIni=&idTipoAutor=&siglaPartidoAutor=&codEstado=&datApresentacaoFim=&parteNomeAutor=&sigla=PL
Status: 200


In [36]:
tree = xml2dict.parse(response.content)


In [37]:
tree.keys()

[u'proposicoes']

In [38]:
proposicoes = tree['proposicoes']
[x for x in proposicoes.keys()]

[u'proposicao']

In [39]:
proposicoes['proposicao'][0].keys()

[u'id',
 u'nome',
 u'tipoProposicao',
 u'numero',
 u'ano',
 u'orgaoNumerador',
 u'datApresentacao',
 u'txtEmenta',
 u'txtExplicacaoEmenta',
 u'regime',
 u'apreciacao',
 u'autor1',
 u'qtdAutores',
 u'ultimoDespacho',
 u'situacao',
 u'indGenero',
 u'qtdOrgaosComEstado']

In [82]:
proposicao = proposicoes['proposicao']

## Nosso resultado tem dicionários aninhados, segue o algoritmo que resolve esse problema

In [83]:
new_dict = {}
chaves_to_delete = []

for prop in proposicao:
    for item in prop:
        if isinstance(prop[item], OrderedDict):
            for x in prop[item].items():
                new_dict[item+'_'+x[0]] = x[1]
            chaves_to_delete.append(item)
            prop.update(new_dict)

In [84]:
for prop in proposicao:
    for chave in prop.keys():
        if chave in chaves_to_delete:
            del prop[chave]

In [103]:
proposicao_df = pd.DataFrame(proposicao)
proposicao_df.head()
print(len(proposicao_df))

2299


In [104]:
shape = proposicao_df.shape
print('%s linhas, %s colunas' % (shape[0], shape[1]))

2299 linhas, 33 colunas


In [105]:
proposicao_df.columns

Index([u'id', u'nome', u'numero', u'ano', u'datApresentacao', u'txtEmenta',
       u'txtExplicacaoEmenta', u'qtdAutores', u'indGenero',
       u'qtdOrgaosComEstado', u'tipoProposicao_id', u'tipoProposicao_sigla',
       u'tipoProposicao_nome', u'orgaoNumerador_sigla', u'orgaoNumerador_nome',
       u'orgaoNumerador_id', u'regime_txtRegime', u'regime_codRegime',
       u'apreciacao_txtApreciacao', u'apreciacao_id',
       u'autor1_txtSiglaPartido', u'autor1_idecadastro',
       u'autor1_txtNomeAutor', u'autor1_codPartido', u'autor1_txtSiglaUF',
       u'ultimoDespacho_datDespacho', u'ultimoDespacho_txtDespacho',
       u'situacao_id', u'situacao_descricao',
       u'situacao_principal_proposicaoPrincipal',
       u'situacao_principal_codProposicaoPrincipal',
       u'situacao_orgao_codOrgaoEstado', u'situacao_orgao_siglaOrgaoEstado'],
      dtype='object')

In [115]:
print(proposicao_df.columns)
proposicao_clean = proposicao_df[pd.notnull(proposicao_df['autor1_idecadastro'])]

Index([u'id', u'nome', u'numero', u'ano', u'datApresentacao', u'txtEmenta',
       u'txtExplicacaoEmenta', u'qtdAutores', u'indGenero',
       u'qtdOrgaosComEstado', u'tipoProposicao_id', u'tipoProposicao_sigla',
       u'tipoProposicao_nome', u'orgaoNumerador_sigla', u'orgaoNumerador_nome',
       u'orgaoNumerador_id', u'regime_txtRegime', u'regime_codRegime',
       u'apreciacao_txtApreciacao', u'apreciacao_id',
       u'autor1_txtSiglaPartido', u'autor1_idecadastro',
       u'autor1_txtNomeAutor', u'autor1_codPartido', u'autor1_txtSiglaUF',
       u'ultimoDespacho_datDespacho', u'ultimoDespacho_txtDespacho',
       u'situacao_id', u'situacao_descricao',
       u'situacao_principal_proposicaoPrincipal',
       u'situacao_principal_codProposicaoPrincipal',
       u'situacao_orgao_codOrgaoEstado', u'situacao_orgao_siglaOrgaoEstado'],
      dtype='object')


In [118]:
proposicao_clean.to_csv('../data/proposicoes_PL.csv', sep=',', encoding='utf-8')

### Podemos ver que nosso dataset possui a relação que precisávamos *( deputados <-> proposições )*

In [4]:
proposicoes = pd.read_csv('../data/proposicoes_PL.csv')
p[['autor1_idecadastro', 'autor1_txtNomeAutor']].head()

,autor1_idecadastro,autor1_txtNomeAutor
0,178828,Evandro Gussi
1,195866,Vaidon Oliveira
2,74787,Milton Monti
3,141401,Carlos Bezerra
4,74399,Onyx Lorenzoni


In [5]:
proposicoes.head()

,Unnamed: 0,id,nome,numero,ano,datApresentacao,txtEmenta,txtExplicacaoEmenta,qtdAutores,indGenero,...,autor1_codPartido,autor1_txtSiglaUF,ultimoDespacho_datDespacho,ultimoDespacho_txtDespacho,situacao_id,situacao_descricao,situacao_principal_proposicaoPrincipal,situacao_principal_codProposicaoPrincipal,situacao_orgao_codOrgaoEstado,situacao_orgao_siglaOrgaoEstado
0,0,2162285,PL 9086/2017,9086,2017,14/11/2017 14:58:00,Dispõe sobre a Política Nacional de Biocombust...,Altera a Lei nº 9.478 de 1997.,1,o,...,140,SP,NaN,NaN,905.0,Aguardando Despacho do Presidente da Câmara do...,NaN,0,100001.0,SECAP(SGM)
1,2,2162215,PL 9084/2017,9084,2017,10/11/2017 17:16:00,"Altera a Lei nº 9.427, de 26 de dezembro de 19...",NaN,1,o,...,222,CE,NaN,NaN,905.0,Aguardando Despacho do Presidente da Câmara do...,NaN,0,100001.0,SECAP(SGM)
2,3,2162212,PL 9083/2017,9083,2017,10/11/2017 15:28:00,"Denomina ""Agência Sergio da Fonseca"" a agência...",NaN,1,o,...,168,SP,NaN,NaN,905.0,Aguardando Despacho do Presidente da Câmara do...,NaN,0,100001.0,SECAP(SGM)
3,4,2162161,PL 9082/2017,9082,2017,10/11/2017 09:48:00,Acrescenta novo § 5º ao art. 22 da Lei nº 11.1...,NaN,1,o,...,111,MT,NaN,NaN,905.0,Aguardando Despacho do Presidente da Câmara do...,NaN,0,100001.0,SECAP(SGM)
4,5,2162132,PL 9081/2017,9081,2017,9/11/2017 17:55:00,"Altera a Lei nº 7.102, de 20 de junho de 1983,...",NaN,1,o,...,202,RS,NaN,NaN,905.0,Aguardando Despacho do Presidente da Câmara do...,NaN,0,100001.0,SECAP(SGM)


In [12]:
[x for x in proposicoes.columns]

['Unnamed: 0',
 'id',
 'nome',
 'numero',
 'ano',
 'datApresentacao',
 'txtEmenta',
 'txtExplicacaoEmenta',
 'qtdAutores',
 'indGenero',
 'qtdOrgaosComEstado',
 'tipoProposicao_id',
 'tipoProposicao_sigla',
 'tipoProposicao_nome',
 'orgaoNumerador_sigla',
 'orgaoNumerador_nome',
 'orgaoNumerador_id',
 'regime_txtRegime',
 'regime_codRegime',
 'apreciacao_txtApreciacao',
 'apreciacao_id',
 'autor1_txtSiglaPartido',
 'autor1_idecadastro',
 'autor1_txtNomeAutor',
 'autor1_codPartido',
 'autor1_txtSiglaUF',
 'ultimoDespacho_datDespacho',
 'ultimoDespacho_txtDespacho',
 'situacao_id',
 'situacao_descricao',
 'situacao_principal_proposicaoPrincipal',
 'situacao_principal_codProposicaoPrincipal',
 'situacao_orgao_codOrgaoEstado',
 'situacao_orgao_siglaOrgaoEstado']

### Deletando coluna *Unnamed: 0*

In [28]:
proposicoes.drop('Unnamed: 0', axis=1, inplace=True)

In [29]:
proposicoes.columns

Index([u'id', u'nome', u'numero', u'ano', u'datApresentacao', u'txtEmenta',
       u'txtExplicacaoEmenta', u'qtdAutores', u'indGenero',
       u'qtdOrgaosComEstado', u'tipoProposicao_id', u'tipoProposicao_sigla',
       u'tipoProposicao_nome', u'orgaoNumerador_sigla', u'orgaoNumerador_nome',
       u'orgaoNumerador_id', u'regime_txtRegime', u'regime_codRegime',
       u'apreciacao_txtApreciacao', u'apreciacao_id',
       u'autor1_txtSiglaPartido', u'autor1_idecadastro',
       u'autor1_txtNomeAutor', u'autor1_codPartido', u'autor1_txtSiglaUF',
       u'ultimoDespacho_datDespacho', u'ultimoDespacho_txtDespacho',
       u'situacao_id', u'situacao_descricao',
       u'situacao_principal_proposicaoPrincipal',
       u'situacao_principal_codProposicaoPrincipal',
       u'situacao_orgao_codOrgaoEstado', u'situacao_orgao_siglaOrgaoEstado'],
      dtype='object')

#### Colunas do tipo *autor1_** são ruins para serem trabalhadas, portanto, temos que consertar isso

In [50]:
columns = list(proposicoes.columns)
keys_to_replaced = []
results = []
for name in columns:
        if "autor1" in name:
            index = columns.index(name)
            columns[index] = name.split("_")[1]

print('-- Lista atualizada --')
print(columns)

-- Lista atualizada --
['id', 'nome', 'numero', 'ano', 'datApresentacao', 'txtEmenta', 'txtExplicacaoEmenta', 'qtdAutores', 'indGenero', 'qtdOrgaosComEstado', 'tipoProposicao_id', 'tipoProposicao_sigla', 'tipoProposicao_nome', 'orgaoNumerador_sigla', 'orgaoNumerador_nome', 'orgaoNumerador_id', 'regime_txtRegime', 'regime_codRegime', 'apreciacao_txtApreciacao', 'apreciacao_id', 'txtSiglaPartido', 'idecadastro', 'txtNomeAutor', 'codPartido', 'txtSiglaUF', 'ultimoDespacho_datDespacho', 'ultimoDespacho_txtDespacho', 'situacao_id', 'situacao_descricao', 'situacao_principal_proposicaoPrincipal', 'situacao_principal_codProposicaoPrincipal', 'situacao_orgao_codOrgaoEstado', 'situacao_orgao_siglaOrgaoEstado']


In [51]:
proposicoes.columns = columns

### Nossas colunas problemáticas foram corrigidas :)

In [54]:
list(proposicoes.columns)

['id',
 'nome',
 'numero',
 'ano',
 'datApresentacao',
 'txtEmenta',
 'txtExplicacaoEmenta',
 'qtdAutores',
 'indGenero',
 'qtdOrgaosComEstado',
 'tipoProposicao_id',
 'tipoProposicao_sigla',
 'tipoProposicao_nome',
 'orgaoNumerador_sigla',
 'orgaoNumerador_nome',
 'orgaoNumerador_id',
 'regime_txtRegime',
 'regime_codRegime',
 'apreciacao_txtApreciacao',
 'apreciacao_id',
 'txtSiglaPartido',
 'idecadastro',
 'txtNomeAutor',
 'codPartido',
 'txtSiglaUF',
 'ultimoDespacho_datDespacho',
 'ultimoDespacho_txtDespacho',
 'situacao_id',
 'situacao_descricao',
 'situacao_principal_proposicaoPrincipal',
 'situacao_principal_codProposicaoPrincipal',
 'situacao_orgao_codOrgaoEstado',
 'situacao_orgao_siglaOrgaoEstado']

## Vamos tentar limpar nossos dados, retirar as colunas que são "redundantes" ou que não nos interessam no momento

In [60]:
proposicoes[['tipoProposicao_id', 'tipoProposicao_nome', 'tipoProposicao_sigla']].head()

,tipoProposicao_id,tipoProposicao_nome,tipoProposicao_sigla
0,139,Projeto de Lei,PL
1,139,Projeto de Lei,PL
2,139,Projeto de Lei,PL
3,139,Projeto de Lei,PL
4,139,Projeto de Lei,PL


In [62]:
proposicoes.drop(['tipoProposicao_id', 'tipoProposicao_nome'], axis=1, inplace=True)

In [63]:
proposicoes.columns

Index([u'id', u'nome', u'numero', u'ano', u'datApresentacao', u'txtEmenta',
       u'txtExplicacaoEmenta', u'qtdAutores', u'indGenero',
       u'qtdOrgaosComEstado', u'tipoProposicao_sigla', u'orgaoNumerador_sigla',
       u'orgaoNumerador_nome', u'orgaoNumerador_id', u'regime_txtRegime',
       u'regime_codRegime', u'apreciacao_txtApreciacao', u'apreciacao_id',
       u'txtSiglaPartido', u'idecadastro', u'txtNomeAutor', u'codPartido',
       u'txtSiglaUF', u'ultimoDespacho_datDespacho',
       u'ultimoDespacho_txtDespacho', u'situacao_id', u'situacao_descricao',
       u'situacao_principal_proposicaoPrincipal',
       u'situacao_principal_codProposicaoPrincipal',
       u'situacao_orgao_codOrgaoEstado', u'situacao_orgao_siglaOrgaoEstado'],
      dtype='object')

In [64]:
proposicoes[['orgaoNumerador_sigla', 'orgaoNumerador_nome', 'orgaoNumerador_id']].head()

,orgaoNumerador_sigla,orgaoNumerador_nome,orgaoNumerador_id
0,PLEN,PLENÁRIO,180
1,PLEN,PLENÁRIO,180
2,PLEN,PLENÁRIO,180
3,PLEN,PLENÁRIO,180
4,PLEN,PLENÁRIO,180


In [65]:
proposicoes.drop(['orgaoNumerador_sigla', 'orgaoNumerador_id'], axis=1, inplace=True)

In [66]:
list(proposicoes.columns)

['id',
 'nome',
 'numero',
 'ano',
 'datApresentacao',
 'txtEmenta',
 'txtExplicacaoEmenta',
 'qtdAutores',
 'indGenero',
 'qtdOrgaosComEstado',
 'tipoProposicao_sigla',
 'orgaoNumerador_nome',
 'regime_txtRegime',
 'regime_codRegime',
 'apreciacao_txtApreciacao',
 'apreciacao_id',
 'txtSiglaPartido',
 'idecadastro',
 'txtNomeAutor',
 'codPartido',
 'txtSiglaUF',
 'ultimoDespacho_datDespacho',
 'ultimoDespacho_txtDespacho',
 'situacao_id',
 'situacao_descricao',
 'situacao_principal_proposicaoPrincipal',
 'situacao_principal_codProposicaoPrincipal',
 'situacao_orgao_codOrgaoEstado',
 'situacao_orgao_siglaOrgaoEstado']

In [67]:
proposicoes[['regime_txtRegime', 'regime_codRegime']].head()

,regime_txtRegime,regime_codRegime
0,.,99
1,.,99
2,.,99
3,.,99
4,.,99


In [68]:
proposicoes.drop(['regime_txtRegime', 'regime_codRegime'], axis=1, inplace=True)

In [69]:
list(proposicoes.columns)

['id',
 'nome',
 'numero',
 'ano',
 'datApresentacao',
 'txtEmenta',
 'txtExplicacaoEmenta',
 'qtdAutores',
 'indGenero',
 'qtdOrgaosComEstado',
 'tipoProposicao_sigla',
 'orgaoNumerador_nome',
 'apreciacao_txtApreciacao',
 'apreciacao_id',
 'txtSiglaPartido',
 'idecadastro',
 'txtNomeAutor',
 'codPartido',
 'txtSiglaUF',
 'ultimoDespacho_datDespacho',
 'ultimoDespacho_txtDespacho',
 'situacao_id',
 'situacao_descricao',
 'situacao_principal_proposicaoPrincipal',
 'situacao_principal_codProposicaoPrincipal',
 'situacao_orgao_codOrgaoEstado',
 'situacao_orgao_siglaOrgaoEstado']

In [70]:
proposicoes[['apreciacao_txtApreciacao', 'apreciacao_id']].head()

,apreciacao_txtApreciacao,apreciacao_id
0,.,99
1,.,99
2,.,99
3,.,99
4,.,99


In [71]:
proposicoes.drop(['apreciacao_txtApreciacao', 'apreciacao_id'], axis=1, inplace=True)

In [72]:
proposicoes.drop(['ultimoDespacho_datDespacho', 'ultimoDespacho_txtDespacho', 'situacao_id', 'situacao_descricao', 'situacao_principal_proposicaoPrincipal', 'situacao_principal_codProposicaoPrincipal', 'situacao_orgao_codOrgaoEstado', 'situacao_orgao_siglaOrgaoEstado'], axis=1, inplace=True)

In [74]:
list(proposicoes.columns)

['id',
 'nome',
 'numero',
 'ano',
 'datApresentacao',
 'txtEmenta',
 'txtExplicacaoEmenta',
 'qtdAutores',
 'indGenero',
 'qtdOrgaosComEstado',
 'tipoProposicao_sigla',
 'orgaoNumerador_nome',
 'txtSiglaPartido',
 'idecadastro',
 'txtNomeAutor',
 'codPartido',
 'txtSiglaUF']

In [76]:
proposicoes.drop(['indGenero'], axis=1, inplace=True)

In [87]:
list(proposicoes.columns)

['id',
 'nome',
 'numero',
 'ano',
 'datApresentacao',
 'txtEmenta',
 'txtExplicacaoEmenta',
 'tipoProposicao_sigla',
 'orgaoNumerador_nome',
 'txtSiglaPartido',
 'idecadastro',
 'txtNomeAutor',
 'txtSiglaUF']

In [85]:
proposicoes.drop('qtdOrgaosComEstado', axis=1, inplace=True)

In [88]:
proposicoes['idecadastro'].head()

0    178828
1    195866
2     74787
3    141401
4     74399
Name: idecadastro, dtype: int64

### Salvando em um CSV

In [55]:
proposicoes.to_csv('../data/proposicoes_PL.csv', sep=',')